In [1]:
import pandas
import numpy
import matplotlib.pyplot as plt
from Bio import SeqIO
import re
import coralme
plt.rcParams['svg.fonttype'] = 'none'
pandas.set_option('display.max_colwidth', None)
%load_ext autoreload
%autoreload 2

## Naive model

In [2]:
model = coralme.io.pickle.load_pickle_me_model("./pputidame/MEModel-step3-pputida-TS.pkl")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-03
Read LP format model from file /tmp/tmpizqvpwxb.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpsiadl4x7.lp
Reading time = 0.01 seconds
: 2162 rows, 5896 columns, 23426 nonzeros
Read LP format model from file /tmp/tmp7hch5nuf.lp
Reading time = 0.01 seconds
: 2166 rows, 5894 columns, 23388 nonzeros


### M9

In [3]:
for r in model.reactions.query("^EX_"):
    r.lower_bound = 0

In [4]:
medium = pandas.read_csv("./media/M9.txt",index_col=0,header=None,comment="#")[1]
for r,b in medium.items():
    model.reactions.get_by_id(r).lower_bound = -b

In [5]:
model.optimize(max_mu=1.0)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.5000000000000000	Optimal
        2	0.7500000000000000	Not feasible
        3	0.6250000000000000	Optimal
        4	0.6875000000000000	Not feasible
        5	0.6562500000000000	Not feasible
        6	0.6406250000000000	Not feasible
        7	0.6328125000000000	Optimal
        8	0.6367187500000000	Not feasible
        9	0.6347656250000000	Not feasible
       10	0.6337890625000000	Optimal
       11	0.6342773437500000	Optimal
       12	0.6345214843750000	Optimal
       13	0.6346435546875000	Not feasible
       14	0.6345825195312500	Optimal
       15	0.6346130371093750	Optimal
       16	0.6346282958984375	Optimal
       17	0.6346359252929688	Not feasible
       18	0.6346321105957031	Not feasible
       19	0.6346302032470703	Optimal
       20	0.6346311569213867	Optimal


True

In [6]:
from coralme.util.flux_analysis import exchange_single_model,flux_based_reactions
exchange_single_model(model).sort_values("flux",ascending=False).head(20)

,name,reaction,lower_bound,upper_bound,flux
id,,,,,
EX_h2o_e,,1.0 h2o_e <=>,-100,999999.0,41.956630
EX_co2_e,,1.0 co2_e -->,0,999999.0,24.728799
EX_h_e,,1.0 h_e <=>,-100,999999.0,6.539237
biomass_dilution,,1.0 biomass -->,mu,mu,0.634631
EX_ac_e,,1.0 ac_e -->,0,999999.0,0.390282
DM_5drib_c,,1.0 5drib_c -->,0.0,999999.0,0.000431
DM_tripeptide_c,,1.0 tripeptide_c -->,0.0,999999.0,0.000186
DM_amob_c,,1.0 amob_c -->,0.0,999999.0,0.000148
DM_doxopa_c,,1.0 doxopa_c -->,0.0,999999.0,0.000142


## Keff sweep

In [7]:
def set_keff(model,keff):
    for r in model.reactions:
        if not isinstance(r,coralme.core.reaction.MetabolicReaction):
            continue
        r.keff = keff
        r.update()

In [18]:
GrowthRates = {}
for keff in range(15,40,5):
    print(keff)
    set_keff(model,keff)
    model.optimize(max_mu=1.0)
    GrowthRates[keff] = model.solution.fluxes["biomass_dilution"]
GrowthRates = pandas.Series(GrowthRates)

15
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.5000000000000000	Not feasible
        2	0.2500000000000000	Optimal
        3	0.3750000000000000	Not feasible
        4	0.3125000000000000	Not feasible
        5	0.2812500000000000	Optimal
        6	0.2968750000000000	Optimal
        7	0.3046875000000000	Optimal
        8	0.3085937500000000	Optimal
        9	0.3105468750000000	Not feasible
       10	0.3095703125000000	Optimal
       11	0.3100585937500000	Optimal
       12	0.3103027343750000	Optimal
       13	0.3104248046875000	Not feasible
       14	0.3103637695312500	Optimal
       15	0.3103942871093750	Optimal
       16	0.3104095458984375	Optimal
       17	0.3104171752929688	Optimal
       18	0.3104209899902344	Not feasible
       19	0.3104190826416016	Optimal
       20	0.3104200363159180	Not feasible
20
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.5000000000000000	Not feasible
     

In [19]:
GrowthRates.to_csv("growth-rates-per-keff.csv")

In [20]:
GrowthRates

15    0.310419
20    0.399662
25    0.483125
30    0.561399
35    0.634631
dtype: float64

In [27]:
m = (GrowthRates[35]-GrowthRates[30])/(35-30) 
b = GrowthRates[30] - m*30
Keff = (0.576-b)/m

In [28]:
Keff

30.99692512045839

In [30]:
set_keff(model,31)
coralme.io.pickle.save_pickle_me_model(model,"./pputidame/MEModel-step3-pputida-Keff.pkl")